In [17]:
import pandas as pd
import numpy as np

In [3]:
df1 = pd.read_csv("../output/stage2/exp005.py/20230922072011_data_dir6_max_len256/valid_predictions.csv")

In [4]:
df2 = pd.read_csv("../output/stage2/exp005.py/20230922082906_data_dir6_max_len384/valid_predictions.csv")

In [5]:
df3 = pd.read_csv("../output/stage2/exp005.py/20230922094119_data_dir6_max_len512/valid_predictions.csv")

In [7]:
def map_at_3(predictions, labels):
    map_sum = 0
    pred = np.argsort(-1*np.array(predictions),axis=1)[:,:3]
    for x,y in zip(pred,labels):
        z = [1/i if y==j else 0 for i,j in zip([1,2,3],x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)

In [9]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
labels = df1["answer"].map(option_to_index).values

In [12]:
df_val = pd.read_parquet("../data/train_parquet/cv_test/6/valid.parquet")

In [15]:
df1["source"] = df_val["source"].values
df2["source"] = df_val["source"].values
df3["source"] = df_val["source"].values

In [18]:
ret = []
for name, df in zip(["m256", "m384", "m512"], [df1, df2, df3]):
    for source, w_df in df.groupby("source"):
        labels = w_df["answer"].map(option_to_index).values
        preds = w_df[["fold0_A", "fold0_B", "fold0_C", "fold0_D", "fold0_E"]].values
        ret.append({
            "name": name,
            "source": source,
            "map3": map_at_3(preds, labels)
        })

In [20]:

df_ret = pd.DataFrame(ret)

In [25]:
df["source"].value_counts()

3                                                                                  1923
10                                                                                 1525
2                                                                                   775
additional_data/MMLU/test.csv                                                       772
4                                                                                   771
5                                                                                   758
6                                                                                   746
additional_data/MMLU/train.csv                                                      588
9                                                                                   462
8                                                                                   398
additional_data/ScienceQA/train.parquet                                             264
additional_data/OpenBookQA-V1-Se

In [23]:
df_ret.pivot_table(index="source", columns="name", values="map3")

name,m256,m384,m512
source,,,
1,0.875817,0.846405,0.843137
10,0.907432,0.913770,0.915519
11,0.913580,0.908642,0.919753
12,0.904077,0.910072,0.914868
2,0.828817,0.831613,0.848172
3,0.737650,0.742936,0.754810
4,0.822957,0.834198,0.843061
5,0.733509,0.759015,0.760334
6,0.815907,0.821492,0.835344


In [30]:
ret = []
for name, df in zip(["m256", "m384", "m512"], [df1, df2, df3]):
    df["over100"] = df["A"].apply(len) > 100
    for source, w_df in df.groupby("over100"):
        labels = w_df["answer"].map(option_to_index).values
        preds = w_df[["fold0_A", "fold0_B", "fold0_C", "fold0_D", "fold0_E"]].values
        ret.append({
            "name": name,
            "source": source,
            "map3": map_at_3(preds, labels)
        })
df_ret = pd.DataFrame(ret)

In [31]:
df_ret.pivot_table(index="source", columns="name", values="map3")

name,m256,m384,m512
source,,,
False,0.791286,0.795089,0.803027
True,0.786910,0.799060,0.812815


In [38]:
ret = []
for name, df in zip(["m256", "m384", "m512"], [df1, df2, df3]):
    for source, w_df in df[df["A"].apply(len) > 100].groupby("source"):
        labels = w_df["answer"].map(option_to_index).values
        preds = w_df[["fold0_A", "fold0_B", "fold0_C", "fold0_D", "fold0_E"]].values
        ret.append({
            "name": name,
            "source": source,
            "map3": map_at_3(preds, labels)
        })
df_ret = pd.DataFrame(ret)
df_ret.pivot_table(index="source", columns="name", values="map3")

name,m256,m384,m512
source,,,
1,0.766667,0.811111,0.744444
10,0.892157,0.872549,0.879902
11,0.916667,1.000000,1.000000
12,1.000000,1.000000,1.000000
2,0.855710,0.858025,0.884259
3,0.743616,0.758109,0.768116
4,0.793478,0.821256,0.835749
5,0.756341,0.783514,0.809783
6,0.827957,0.838710,0.846774


In [39]:
ret = []
for name, df in zip(["m256", "m384", "m512"], [df1, df2, df3]):
    for source, w_df in df[df["A"].apply(len) < 100].groupby("source"):
        labels = w_df["answer"].map(option_to_index).values
        preds = w_df[["fold0_A", "fold0_B", "fold0_C", "fold0_D", "fold0_E"]].values
        ret.append({
            "name": name,
            "source": source,
            "map3": map_at_3(preds, labels)
        })
df_ret = pd.DataFrame(ret)
df_ret.pivot_table(index="source", columns="name", values="map3")

name,m256,m384,m512
source,,,
1,0.921296,0.861111,0.884259
10,0.907828,0.915404,0.916896
11,0.912760,0.903646,0.915365
12,0.902676,0.908759,0.913625
2,0.818645,0.821643,0.834532
3,0.736393,0.739430,0.750526
4,0.828549,0.837852,0.843966
5,0.722810,0.748089,0.741329
6,0.810101,0.813131,0.829966
